In [1]:
# Detecta la board en cada fotograma.
# Va cogiendo los 20 últimos fotogramas de la board y su resultado más común es el state de la board.
# Si se produce un cambio entre los dos últimos states te devuelve el output donde el robot tiene que poner el token (de forma random).
# Si los 3 tokens están en partida, se quita un token de una casilla y se pone en otra.

# ERRORES:
# No diferencia bien entre los tokens verdes y rojos

import cv2
import numpy as np
from collections import defaultdict
import random
import time
from PIL import Image




height, width = 450, 350

# Define the four corners of the tic tac toe board in the original frame

coordinates = [[694, 665], [1170, 666], [524, 1048], [1422, 1028]]


src_points = np.float32(coordinates)

# Define the four corners of the destination image
dst_points = np.float32([[0, 0], [width, 0], [0, height], [width, height]])


# Initialize a list that will contain the boards
boards = []

# Initialize a list that will contain the states of the board 
results = [[[0, 0, 0], [0, 0, 0], [0, 0, 0]]]

result = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]

# count of how many tokens you have placed on the board
tokens_placed = 0

# check for game_over function

def check_win(board):
    # Check rows
    for row in board:
        if len(set(row)) == 1 and row[0] != 0:
            return True

    # Check columns
    for col in range(3):
        if len(set([board[row][col] for row in range(3)])) == 1 and board[0][col] != 0:
            return True

    # Check diagonals
    if len(set([board[i][i] for i in range(3)])) == 1 and board[0][0] != 0:
        return True
    
    if len(set([board[i][2-i] for i in range(3)])) == 1 and board[0][2] != 0:
        return True

    return False

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    transformed_frame = cv2.warpPerspective(frame, M, (width, height))
    pil_image = Image.fromarray(transformed_frame)
    transformed_frame = pil_image.rotate(180)
    transformed_frame = np.array(transformed_frame)

     # Convert the transformed frame to grayscale
    gray = cv2.cvtColor(transformed_frame, cv2.COLOR_BGR2GRAY)

    # Adjust the Canny edge detection thresholds
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(gray, low_threshold, high_threshold, apertureSize=3)

    # Find contours in the image
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize an empty list to store the coordinates of the squares
    squares = []

    # Loop through the contours
    for cnt in contours:
        # Approximate the contour with a polygon
        approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)
        # If the polygon has four vertices and is convex, it could be a rectangle
        if len(approx) == 4 and cv2.isContourConvex(approx):
            # Calculate the bounding box of the polygon
            x, y, w, h = cv2.boundingRect(approx)
            # Draw a green rectangle around the rectangle
            cv2.rectangle(transformed_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
             # Add the coordinates of the rectangle to the list
            squares.append((x, y, x+w, y+h))

    # Sort the squares from left to right and top to bottom
    squares = sorted(squares, key=lambda c: (c[1] // 124) * 3 + (c[0] // 107))
   # if len(squares)==9:
    #    print(squares)

    # Initialize a 3x3 grid of zeros to represent the tic tac toe board
    board = np.zeros((3, 3), dtype=np.int) 

    # Check if there are exactly 9 squares
    if len(squares) != 9:
        #print("Error: Detected {} squares instead of 9".format(len(squares)))
        exit()
    
    else:
    # Loop through the squares and fill in the corresponding cell of the grid
        for i, square in enumerate(squares):
            row = i // 3
            col = i % 3
            x1, y1, x2, y2 = square
            # Add the square to the grid
            board[row, col] = 0

            # Crop the image to the bounding box of the square
            square_img = transformed_frame[y1:y2, x1:x2]

            # Convert the cropped image to grayscale
            square_gray = cv2.cvtColor(square_img, cv2.COLOR_BGR2GRAY)

            # Aplicar un filtro gaussiano para reducir el ruido
            blur = cv2.GaussianBlur(gray, (5, 5), 0)

            # Apply the Hough circle transform to detect circles
            circles = cv2.HoughCircles(square_gray, cv2.HOUGH_GRADIENT, dp=1, minDist=20, param1=50, param2=12, minRadius=20, maxRadius=30)


            if circles is not None:

                circles = circles[0]
                for (x, y, r) in circles:
                    # Extraer la región del círculo
                    circle_region = square_img[int(y-r):int(y+r), int(x-r):int(x+r)]
                    if not circle_region.any():
                        continue
                    circle_hsv = cv2.cvtColor(circle_region, cv2.COLOR_BGR2HSV)
                    
                    
                    # Definir rangos de color para rojo y verde
                    lower_red = (169, 148, 180)
                    upper_red = (176, 193, 235)
                    
                    lower_green = (55, 99, 158)
                    upper_green = (64, 142, 236)
                    
                    # Segmentar el color del círculo utilizando los rangos de color definidos
                    mask_red = cv2.inRange(circle_hsv, lower_red, upper_red)
                    mask_green = cv2.inRange(circle_hsv, lower_green, upper_green)
                    
                    # Contar los píxeles de cada máscara para determinar el color predominante
                    count_red = cv2.countNonZero(mask_red)
                    count_green = cv2.countNonZero(mask_green)
                    
                    # Imprimir el resultado
                    if count_red > count_green:
                        board[row, col] = 1 
                                            
                    else:
                        board[row, col] = 2 
        
        board_list = board.tolist()
        # print(board_list)

        # Append the all the new boards to a list
        boards.append(board_list)
        #print(len(boards))
        freq = defaultdict(int)

        if len(boards) == 20:
            for board in boards:
                freq[str(board)] += 1

            # find board with highest frequency
            max_board = max(freq, key=freq.get)
            result = eval(max_board)  # convert string representation back to list
            print(result)
            boards = []

            
        # Print the grid
        #print(board)

        

    cv2.imshow('frame', transformed_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.waitKey(0)
cv2.destroyAllWindows()

<ipython-input-1-fc6ba041951c>:108: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  board = np.zeros((3, 3), dtype=np.int)


[[0, 0, 0], [0, 0, 2], [1, 1, 1]]
